# Homework 3. Solution key

In [1]:
import numpy as np

## Q1
Consider the polynomial
\begin{equation}
p\left(x\right)=a_{0}+a_{1}x+a_{2}x^{2}+\ldots+a_{n}x^{n}=\sum_{i=0}^{n}a_{i}x^{i}.
\end{equation}

Write three versions of the function ``poly(x, coef)`` that computes the value of the polynomial above given a point $x$ and a list of coefficients $\left[a_{0},\ldots,a_{n}\right]$.

In [2]:
x, coef = .1, np.random.rand(10000)

Use loops with ``enumerate()`` function.

In [3]:
def poly(x, coef):
    out = 0
    for i, ai in enumerate(coef):
        out += ai * x**i
    return out

print(poly(x, coef))
%timeit poly(x, coef)

0.127020378673
100 loops, best of 3: 6.88 ms per loop


Vectorize the code with the help of ``np.cumsum()``.

In [4]:
def poly(x, coef):
    return np.cumsum(coef * x**np.arange(len(coef)))[-1]

print(poly(x, coef))
%timeit poly(x, coef)

0.127020378673
100 loops, best of 3: 2.74 ms per loop


Use ``np.poly1d()``.

In [5]:
def poly(x, coef):
    return np.poly1d(coef[::-1])(x)

print(poly(x, coef))
%timeit poly(x, coef)

0.127020378673
100 loops, best of 3: 17.8 ms per loop


## Q2

Let ``probs`` be a NumPy array of length $n$ with ``probs.sum()`` equal to 1, so that they can be interpreted as probabilities. We wish to generate a discrete random variable $X$ such that $P\left[X=i\right]=p_{i}$. The standard algorithm is as follows:

- Divide the unit interval $\left[0,1\right]$ into $n$ subintervals $I_{0},I_{1},\ldots,I_{n-1}$ such that the length of $I_{i}$ is $p_{i}$.

- Draw a uniform random variable $U$ on $\left[0,1\right]$ and return the $i$ such that $U\in I_{i}$.

The probability of drawing $i$ is the length of $I_{i}$, which is equal to $p_{i}$. The algorithm can be implemented as follows:

In [6]:
from random import uniform

def sample(probs):
    a = 0.0
    unif = uniform(0, 1)
    for i in range(len(probs)):
        if a < unif <= a + probs[i]:
            return i
        a += probs[i]

def draw(probs, size=1):
    return np.array([sample(probs) for _ in range(size)])

probs = [.1, .2, .7]
out = draw(probs, size=10)

print(out)

%timeit draw(probs, size=1000)

[1 2 0 2 1 2 2 2 2 0]
1000 loops, best of 3: 1.78 ms per loop


Vectorize this code with the help of ``np.searchsorted`` and ``np.cumsum``.

In [7]:
class DiscreteRV(object):
    
    """Discrete random variable.
    
    Attributes
    ----------
    probs
        Probability distribution
        
    Methods
    -------
    draw
        Random number generator
        
    Examples
    --------
    >>> rv = DiscreteRV(probs)
    >>> out = rv.draw(10)
    >>> print(out)
    
    """
    def __init__(self, probs):
        """Initialize the instance.
        
        Parameters
        ----------
        probs : array-like
            Probability distribution
            
        """
        self.probs = probs
        
    def draw(self, size=1):
        """Draw random sample.
        
        Parameters
        ----------
        size : int
            Size of the sample
        
        Returns
        -------
        array
            Random sample
            
        """
        cumprob = np.cumsum(np.cumsum(self.probs))
        return cumprob.searchsorted(np.random.uniform(0, 1, size=size)) 


rv = DiscreteRV(probs)
out = rv.draw(10)
print(out)

%timeit rv.draw(1000)

[2 1 2 2 2 2 0 1 2 2]
10000 loops, best of 3: 46.7 µs per loop
